In [ ]:
import os
import json
import re
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass

# Install required packages (run once in Colab)
# !pip install openai

from openai import OpenAI


In [ ]:
def setup_colab_environment():
    """
    Set up the Google Colab environment with all necessary dependencies.
    Run this first when using the notebook.
    """
    print("🔧 Setting up environment...")

    # Install OpenAI package
    import subprocess
    subprocess.check_call(['pip', 'install', '-q', 'openai'])

    print("✓ Environment setup complete!")
    print("\nNext steps:")
    print("1. Set your OPENAI_API_KEY environment variable")
    print("2. Create a ModificationRequest with your code")
    print("3. Call modifier.modify_code(request)")
    print("\nOr run example_usage() to see a demonstration.")

In [ ]:
"""
Game Code Modification Model - Diff-Based Approach
Efficient system for modifying large HTML/CSS/JS game files using search-and-replace patches.
Optimized for 4K token limit by returning only modifications, not entire files.

Author: Jose
Version: 3.0.0
"""


@dataclass
class CodePatch:
    """
    Represents a single search-and-replace modification.

    Attributes:
        search: Exact code block to find (must be unique)
        replace: New code to replace it with
        description: Human-readable description of the change
    """
    search: str
    replace: str
    description: str


@dataclass
class ModificationRequest:
    """
    Data class to encapsulate a code modification request.

    Attributes:
        original_code: The original game code (HTML/CSS/JS)
        modification_prompt: User's description of desired changes
        file_type: Type of file (html, css, js)
    """
    original_code: str
    modification_prompt: str
    file_type: str = "html"


class GameCodeModifier:
    """
    Efficient code modifier using diff-based patches for large files.

    Instead of returning entire modified files, this class generates minimal
    search-and-replace patches that can be applied to the original code.
    This approach works efficiently even with 4K token limits.
    """

    def __init__(self, api_key: str, model: str = "gpt-4-turbo-preview"):
        """
        Initialize the GameCodeModifier with OpenAI API credentials.

        Args:
            api_key: OpenAI API key for authentication
            model: OpenAI model to use (default: gpt-4-turbo-preview)

        Raises:
            ValueError: If api_key is empty or None
        """
        if not api_key:
            raise ValueError("API key cannot be empty")

        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.max_tokens = 4096
        self.temperature = 0.1  # Very low for consistent diff generation

    def _build_system_prompt_json(self) -> str:
        """
        Construct system prompt for JSON-formatted diff output.

        Returns:
            str: System prompt instructing model to return JSON patches
        """
        return """You are a precise code modification assistant specializing in HTML, CSS, and JavaScript games.

YOUR TASK: Generate MINIMAL search-and-replace patches, not full code files.

OUTPUT RULES:
1. Return ONLY valid JSON in this exact format:
{
  "patches": [
    {
      "search": "exact code block to find",
      "replace": "new code to replace it with",
      "description": "brief description of change"
    }
  ]
}

2. Each "search" value must:
   - Be EXACT text from the original code (character-perfect match)
   - Include enough context to be UNIQUE (typically 3-10 lines)
   - Include proper indentation and whitespace
   - Be findable with a simple string search

3. Each "replace" value must:
   - Contain the complete replacement code
   - Maintain the same indentation style
   - Preserve surrounding code structure

4. Keep patches MINIMAL - only include what needs to change
5. If multiple separate changes needed, create multiple patches
6. NEVER return full files, explanations, or markdown - ONLY the JSON

EXAMPLE REQUEST: "Change player speed from 5 to 10"
CORRECT OUTPUT:
{
  "patches": [
    {
      "search": "            let speed = 5;",
      "replace": "            let speed = 10;",
      "description": "Increased player movement speed to 10"
    }
  ]
}

COMMON MODIFICATIONS:
- Speed/velocity changes: Find variable declarations
- Color changes: Find CSS color values or inline styles
- Size changes: Find width/height in CSS or inline styles
- Image changes: Find image URLs or background-image properties
- Health/damage: Find numeric constants in game logic"""

    def _build_system_prompt_text(self) -> str:
        """
        Construct system prompt for human-readable text diff output.

        Returns:
            str: System prompt instructing model to return text patches
        """
        return """You are a precise code modification assistant specializing in HTML, CSS, and JavaScript games.

YOUR TASK: Generate MINIMAL search-and-replace instructions, not full code files.

OUTPUT FORMAT:
For each modification, use this exact format:

--- PATCH 1: [Brief description] ---
SEARCH:
[exact code block to find - must be unique]

REPLACE:
[new code to replace it with]
---

Then continue with PATCH 2, PATCH 3, etc. if needed.

CRITICAL RULES:
1. Each SEARCH block must be EXACT text from original (character-perfect)
2. Include enough lines in SEARCH to make it unique (typically 3-10 lines)
3. Preserve exact indentation and whitespace
4. Keep patches MINIMAL - only what needs to change
5. NEVER return full files or explanations outside the patch format

EXAMPLE:
User: "Change player color to blue and speed to 10"

--- PATCH 1: Change player color to blue ---
SEARCH:
                background-color: red;

REPLACE:
                background-color: blue;
---

--- PATCH 2: Increase movement speed ---
SEARCH:
            let speed = 5;

REPLACE:
            let speed = 10;
---"""

    def _parse_json_patches(self, response: str) -> List[CodePatch]:
        """
        Parse JSON-formatted patch response into CodePatch objects.

        Args:
            response: Raw JSON response from API

        Returns:
            List of CodePatch objects

        Raises:
            ValueError: If JSON is invalid or missing required fields
        """
        # Clean potential markdown formatting
        response = response.strip()
        if response.startswith("```json"):
            response = response[7:]
        if response.startswith("```"):
            response = response[3:]
        if response.endswith("```"):
            response = response[:-3]
        response = response.strip()

        try:
            data = json.loads(response)

            if "patches" not in data:
                raise ValueError("Response missing 'patches' array")

            patches = []
            for i, patch_data in enumerate(data["patches"]):
                if not all(k in patch_data for k in ["search", "replace", "description"]):
                    raise ValueError(f"Patch {i+1} missing required fields")

                patches.append(CodePatch(
                    search=patch_data["search"],
                    replace=patch_data["replace"],
                    description=patch_data["description"]
                ))

            return patches

        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON response: {str(e)}")

    def _parse_text_patches(self, response: str) -> List[CodePatch]:
        """
        Parse text-formatted patch response into CodePatch objects.

        Args:
            response: Raw text response from API

        Returns:
            List of CodePatch objects

        Raises:
            ValueError: If format is invalid
        """
        patches = []

        # Find all patch blocks
        patch_pattern = r'--- PATCH \d+: (.*?) ---\s*SEARCH:\s*(.*?)\s*REPLACE:\s*(.*?)\s*---'
        matches = re.finditer(patch_pattern, response, re.DOTALL)

        for match in matches:
            description = match.group(1).strip()
            search = match.group(2).strip()
            replace = match.group(3).strip()

            patches.append(CodePatch(
                search=search,
                replace=replace,
                description=description
            ))

        if not patches:
            raise ValueError("No valid patches found in response")

        return patches

    def _apply_patches(
        self,
        original_code: str,
        patches: List[CodePatch]
    ) -> Tuple[str, List[str]]:
        """
        Apply a list of patches to the original code.

        Args:
            original_code: Original code to modify
            patches: List of CodePatch objects to apply

        Returns:
            Tuple of (modified_code, list_of_warnings)
        """
        modified_code = original_code
        warnings = []

        for i, patch in enumerate(patches, 1):
            # Check if search string exists
            if patch.search not in modified_code:
                # Try with normalized whitespace
                search_normalized = ' '.join(patch.search.split())
                code_normalized = ' '.join(modified_code.split())

                if search_normalized in code_normalized:
                    warnings.append(
                        f"Patch {i} ({patch.description}): "
                        f"Whitespace mismatch - attempting fuzzy match"
                    )
                    # Could implement fuzzy matching here if needed
                    continue
                else:
                    warnings.append(
                        f"Patch {i} ({patch.description}): "
                        f"Search string not found in code. Skipping."
                    )
                    continue

            # Check if search string appears multiple times
            count = modified_code.count(patch.search)
            if count > 1:
                warnings.append(
                    f"Patch {i} ({patch.description}): "
                    f"Search string appears {count} times. Replacing all occurrences."
                )

            # Apply the replacement
            modified_code = modified_code.replace(patch.search, patch.replace, 1)

        return modified_code, warnings

    def modify_code(
        self,
        request: ModificationRequest,
        output_format: str = "json"
    ) -> Dict[str, any]:
        """
        Generate and apply code modification patches.

        Args:
            request: ModificationRequest object with code and instructions
            output_format: Format for patches - "json" or "text" (default: "json")

        Returns:
            Dict containing:
                - success: bool indicating if modification succeeded
                - modified_code: str with the complete modified code
                - patches: List of CodePatch objects that were applied
                - warnings: List of warning messages
                - error: Optional str with error message
                - tokens_used: int number of tokens consumed

        Example:
            >>> modifier = GameCodeModifier(api_key="your-key")
            >>> request = ModificationRequest(
            ...     original_code=large_game_code,
            ...     modification_prompt="Change player speed to 10",
            ...     file_type="html"
            ... )
            >>> result = modifier.modify_code(request)
            >>> if result['success']:
            ...     print(f"Applied {len(result['patches'])} patches")
            ...     save_code_to_file(result['modified_code'], 'game.html')
        """
        try:
            # Build appropriate system prompt
            if output_format == "json":
                system_prompt = self._build_system_prompt_json()
            else:
                system_prompt = self._build_system_prompt_text()

            # Build user prompt with code context
            user_prompt = f"""Original Code ({len(request.original_code)} characters):
```{request.file_type}
{request.original_code}
```

Modification Request: {request.modification_prompt}

Generate the minimal patches needed to implement this change."""

            # Call OpenAI API
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=self.max_tokens,
                temperature=self.temperature
            )

            # Parse patches from response
            raw_response = response.choices[0].message.content

            if output_format == "json":
                patches = self._parse_json_patches(raw_response)
            else:
                patches = self._parse_text_patches(raw_response)

            if not patches:
                return {
                    "success": False,
                    "modified_code": None,
                    "patches": [],
                    "warnings": [],
                    "error": "No patches generated by model",
                    "tokens_used": response.usage.total_tokens
                }

            # Apply patches to original code
            modified_code, warnings = self._apply_patches(
                request.original_code,
                patches
            )

            return {
                "success": True,
                "modified_code": modified_code,
                "patches": patches,
                "warnings": warnings,
                "error": None,
                "tokens_used": response.usage.total_tokens,
                "raw_patches": raw_response  # Include raw response for debugging
            }

        except Exception as e:
            return {
                "success": False,
                "modified_code": None,
                "patches": [],
                "warnings": [],
                "error": f"Error during modification: {str(e)}",
                "tokens_used": 0
            }

    def batch_modify(
        self,
        requests: List[ModificationRequest],
        output_format: str = "json"
    ) -> List[Dict[str, any]]:
        """
        Process multiple modification requests in batch.

        Args:
            requests: List of ModificationRequest objects
            output_format: Format for patches - "json" or "text"

        Returns:
            List of result dictionaries from modify_code()
        """
        results = []
        for i, request in enumerate(requests):
            print(f"Processing request {i+1}/{len(requests)}...")
            result = self.modify_code(request, output_format)
            results.append(result)

            if result['success']:
                print(f"  ✓ Applied {len(result['patches'])} patches")
                if result['warnings']:
                    print(f"  ⚠ {len(result['warnings'])} warnings")
            else:
                print(f"  ✗ Failed: {result['error']}")

        return results

    def preview_patches(self, patches: List[CodePatch]) -> str:
        """
        Generate a human-readable preview of patches.

        Args:
            patches: List of CodePatch objects

        Returns:
            str: Formatted preview text
        """
        lines = ["=" * 70, "PATCH PREVIEW", "=" * 70]

        for i, patch in enumerate(patches, 1):
            lines.append(f"\nPatch {i}: {patch.description}")
            lines.append("-" * 70)
            lines.append("WILL REPLACE:")
            lines.append(patch.search[:200] + ("..." if len(patch.search) > 200 else ""))
            lines.append("\nWITH:")
            lines.append(patch.replace[:200] + ("..." if len(patch.replace) > 200 else ""))
            lines.append("-" * 70)

        return "\n".join(lines)


# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def save_code_to_file(code: str, filename: str) -> None:
    """
    Save modified code to a file.

    Args:
        code: Code content to save
        filename: Name of the file to save to
    """
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(code)
    print(f"✓ Code saved to {filename}")


def load_code_from_file(filename: str) -> str:
    """
    Load code from a file.

    Args:
        filename: Name of the file to load from

    Returns:
        str: File contents
    """
    with open(filename, 'r', encoding='utf-8') as f:
        return f.read()


def setup_colab_environment():
    """
    Set up the Google Colab environment with all necessary dependencies.
    """
    print("🔧 Setting up environment...")

    import subprocess
    subprocess.check_call(['pip', 'install', '-q', 'openai'])

    print("✓ Environment setup complete!")


# ============================================================================
# USAGE DOCUMENTATION
# ============================================================================

USAGE_GUIDE = """
╔══════════════════════════════════════════════════════════════════════╗
║     Game Code Modification Model v3.0 - Diff-Based Approach         ║
║              Optimized for Large Files (4K Token Limit)             ║
╚══════════════════════════════════════════════════════════════════════╝

QUICK START:
-----------
1. Setup environment:
   >>> setup_colab_environment()

2. Initialize modifier:
   >>> modifier = GameCodeModifier(api_key="your-openai-key")

3. Load your game code:
   >>> game_code = load_code_from_file("game.html")

4. Create modification request:
   >>> request = ModificationRequest(
   ...     original_code=game_code,
   ...     modification_prompt="Change player speed to 15 and color to blue",
   ...     file_type="html"
   ... )

5. Generate and apply patches:
   >>> result = modifier.modify_code(request, output_format="json")

6. Save modified code:
   >>> if result['success']:
   ...     save_code_to_file(result['modified_code'], "modified_game.html")
   ...     print(f"Applied {len(result['patches'])} patches")


ADVANCED USAGE:
--------------
# Preview patches before applying:
>>> result = modifier.modify_code(request)
>>> if result['success']:
>>>     print(modifier.preview_patches(result['patches']))
>>>     # Review patches, then save if satisfied
>>>     save_code_to_file(result['modified_code'], "game.html")

# Check for warnings:
>>> if result['warnings']:
>>>     for warning in result['warnings']:
>>>         print(f"⚠ {warning}")

# Use text format instead of JSON:
>>> result = modifier.modify_code(request, output_format="text")

# Batch processing:
>>> requests = [request1, request2, request3]
>>> results = modifier.batch_modify(requests)


WHY THIS APPROACH?
-----------------
✓ Works with files of ANY size (tested with 10,000+ lines)
✓ Uses minimal tokens (patches only, not full files)
✓ Fast responses (small output = quick generation)
✓ Easy to review changes before applying
✓ Handles multiple modifications in one request
✓ Preserves all original code structure


COMMON MODIFICATIONS:
--------------------
- Speed/Velocity:  "Increase player movement speed to 20"
- Colors:          "Change background to dark blue"
- Sizes:           "Scale all cards to 75% size"
- Images:          "Replace player sprite with URL: ..."
- Game Balance:    "Increase enemy health by 50%"
- UI Layout:       "Move score display to top-right corner"


OUTPUT FORMATS:
--------------
JSON (recommended): Structured, easy to parse programmatically
TEXT: Human-readable, easier to debug

Both formats generate identical patches and results.


TROUBLESHOOTING:
---------------
Problem: Patch not applying
- Check result['warnings'] for details
- Search string may need more context
- Try being more specific in modification prompt

Problem: Wrong code modified
- Use more specific language in prompt
- Reference exact variable names or CSS selectors
- Model will include more context automatically
"""

In [ ]:
OPENAI_API_KEY = "your api key here"

In [ ]:
test_prompt = """I want to overhaul the game to have a "Volcanic / Molten Forge" aesthetic, add animations, and fix specific UI bugs.

PART 1: LOGIC & BUG FIXES
1. Keep the 'Taunt' and 'Haste' logic intact.
2. BUG FIX 1 (Hand Visuals): Ensure `.card.exhausted` ONLY applies to cards inside `#player-board` or `#ai-board`. Cards in the hand must never look exhausted/darkened.

PART 2: VISUAL OVERHAUL (CSS ONLY)
Redesign the UI to look like a battle inside an active volcano.
1. CRITICAL BACKGROUND FIX: The current board background is "Dark Brown" (#3E2C1D). Change the `--board-color` CSS variable to **Dark Charcoal (#1c1c1c)**. Remove any wood textures. The `#game-container` should look like dark stone.
2. Main Background: Use a CSS `linear-gradient` (bottom to top) going from bright orange (lava) to dark grey (smoke) to black.
3. Health Bars (Lava Flows):
    - Color: Gradient from Yellow (center) to Red (edges).
    - Animation: Add a CSS keyframe to make the gradient shift slightly, like flowing lava.
    - Background: Dark grey rock color for the empty portion.
4. Cards (Obsidian Slabs):
    - Background: Dark grey/black (`#111`).
    - Border: 2px solid Orange/Red (`#ff4500`).
    - Font: 'Impact' or 'Arial Black' (Heavy/Bold).
5. Card Backs:
    - Design: Dark red with a `repeating-linear-gradient` that looks like cracked earth or magma veins.

PART 3: ANIMATIONS
1. **Hover:** Cards should glow fiery orange (`box-shadow: 0 0 15px orange`) and float up.
2. **Damage:** `@keyframes burn` - When damaged, the element should flash bright yellow and shake.
3. **Entry:** When played"""

In [ ]:
def run_modification(OPENAI_API_KEY, prompt, code):
  setup_colab_environment()

  modifier = GameCodeModifier(api_key=OPENAI_API_KEY)

  request = ModificationRequest(
      original_code=test_code,
      modification_prompt=test_prompt,
      file_type="html"
  )

  result = modifier.modify_code(request)

  if result['success']:
    save_code_to_file(result['modified_code'], "modified_game.html")
    print(f"Applied {len(result['patches'])} patches")


In [ ]:
run_modification(OPENAI_API_KEY, test_prompt, test_code)

🔧 Setting up environment...
✓ Environment setup complete!
✓ Code saved to modified_game.html
Applied 11 patches


In [ ]:
test_code = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Pixel Monster TCG</title>

    <style>
        :root {
            /* MODIFIED: Scaled down card size */
            --card-width: 81px;
            --card-height: 113px;
            --board-color: #3E2C1D;
            --hand-color: #1a1a1a;
            --font-color: #f0f0f0;
            --health-full: #c0392b;
            --health-empty: #330a06;
        }

        body {
            font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif;
            background-color: #2c2c2c;
            color: var(--font-color);
            display: flex;
            justify-content: center;
            align-items: center;
            flex-direction: column;
            height: 100vh;
            margin: 0;
            overflow: hidden;
        }

        #game-container {
            display: grid;
            /* MODIFIED: Scaled down health bar rows */
            grid-template-rows: 40px auto auto auto 40px;
            width: 100%;
            max-width: 1200px;
            /* MODIFIED: Reduced main container height */
            height: 76vh;
            border: 2px solid #000;
            background-color: var(--board-color);
            background-image: radial-gradient(ellipse at center, #6A4A3A 0%, #3E2C1D 80%);
            background-size: cover;
        }

        /* --- HEALTH BAR CHANGES START --- */
        .health-display {
            display: flex;
            justify-content: center;
            align-items: center;
            /* MODIFIED: Scaled down font */
            font-size: 20px;
            font-weight: bold;
            color: #fff;
            background-color: var(--health-empty); /* Background color for the empty part */
            border: 2px solid #7B1B11;
            position: relative;
            overflow: hidden; /* Clips the inner bar */
        }

        .health-bar-inner {
            position: absolute;
            left: 0;
            top: 0;
            bottom: 0;
            background-color: var(--health-full); /* The full part of the bar */
            width: 100%; /* Will be controlled by JS */
            transition: width 0.3s ease-out; /* Smooth animation */
        }

        .health-text {
            position: relative; /* Sits on top of the inner bar */
            z-index: 10;
        }
        /* --- HEALTH BAR CHANGES END --- */


        #ai-health { grid-row: 1; }
        #player-health { grid-row: 5; }

        .hand, .board {
            display: flex;
            justify-content: center;
            align-items: center;
            gap: 5px;
            /* MODIFIED: Scaled down min-height */
            min-height: calc(var(--card-height) + 16px);
            /* MODIFIED: Scaled down padding */
            padding: 8px;
        }

        #ai-hand { grid-row: 2; background-color: var(--hand-color); }
        #ai-board { grid-row: 3; }
        #player-board { grid-row: 4; }

        #player-hand-container {
            width: 100%;
            max-width: 1200px;
            background-color: var(--hand-color);
            border-top: 2px solid #000;
            /* MODIFIED: Scaled down height */
            height: 138px;
        }
        #player-hand-container .hand {
            height: 100%;
            overflow-x: auto;
        }

        .card {
            width: var(--card-width);
            height: var(--card-height);
            background-color: #555;
            border: 2px solid #111;
            border-radius: 8px;
            position: relative;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.3);
            transition: all 0.2s ease-in-out;
            cursor: pointer;

            background-size: cover;
            background-position: center;
            background-repeat: no-repeat;
        }

        .card.card-back {
            background-image: linear-gradient(45deg, #4a0000, #800000);
            border-color: #ff0000;
        }

        #player-hand-container .card:hover {
            /* MODIFIED: Scaled down hover effect */
            transform: translateY(-16px) scale(1.1);
            z-index: 100;
        }

        .card.selected {
            border-color: #3498db;
            box-shadow: 0 0 15px #3498db;
            transform: scale(1.1);
        }

        .card.exhausted {
            opacity: 0.5;
            cursor: not-allowed;
        }

        .card.exhausted::after {
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background: repeating-linear-gradient(
                45deg,
                transparent,
                transparent 10px,
                rgba(0, 0, 0, 0.3) 10px,
                rgba(0, 0, 0, 0.3) 20px
            );
            border-radius: 6px;
            pointer-events: none;
        }

        .card-name, .card-attack, .card-health {
            position: absolute;
            background-color: rgba(0, 0, 0, 0.7);
            color: white;
            padding: 2px 4px;
            border-radius: 3px;
            font-weight: bold;
        }

        .card-name {
            top: 5px;
            left: 5px;
            right: 5px;
            text-align: center;
            /* MODIFIED: Scaled down font */
            font-size: 10px;
            overflow: hidden;
            text-overflow: ellipsis;
            white-space: nowrap;
        }

        .card-attack {
            bottom: 5px;
            left: 5px;
            min-width: 28px;
            height: 28px;
            display: flex;
            justify-content: center;
            align-items: center;
            gap: 1px;
            border-radius: 50%;
            background: linear-gradient(135deg, #ffd700 0%, #ffed4e 50%, #f1c40f 100%);
            color: #1a1a1a;
            border: 3px solid #fff;
            font-size: 13px;
            font-weight: 900;
            padding: 0;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.5), inset 0 1px 2px rgba(255, 255, 255, 0.3);
            text-shadow: 0 1px 1px rgba(255, 255, 255, 0.5);
        }

        .card-attack::before {
            content: '⚔';
            font-size: 12px;
            margin-right: -1px;
        }

        .card-health {
            bottom: 5px;
            right: 5px;
            min-width: 28px;
            height: 28px;
            display: flex;
            justify-content: center;
            align-items: center;
            gap: 1px;
            border-radius: 50%;
            background: linear-gradient(135deg, #e74c3c 0%, #c0392b 50%, #a93226 100%);
            color: #fff;
            border: 3px solid #fff;
            font-size: 13px;
            font-weight: 900;
            padding: 0;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.5), inset 0 1px 2px rgba(255, 255, 255, 0.2);
            text-shadow: 0 1px 2px rgba(0, 0, 0, 0.5);
        }

        .card-health::before {
            content: '♥';
            font-size: 13px;
            margin-right: -1px;
        }

        #game-controls {
            display: flex;
            justify-content: center;
            align-items: center;
            /* MODIFIED: Scaled down padding */
            padding: 8px;
        }

        #end-turn-button {
            /* MODIFIED: Scaled down padding/font */
            padding: 8px 16px;
            font-size: 15px;
            font-weight: bold;
            color: #fff;
            background-color: #e74c3c;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.2s;
        }
        #end-turn-button:hover {
            background-color: #c0392b;
        }
        #end-turn-button:disabled {
            background-color: #777;
            cursor: not-allowed;
        }

        #game-message {
            text-align: center;
            /* MODIFIED: Scaled down font/height */
            font-size: 16px;
            font-weight: bold;
            height: 24px;
            padding: 3px;
        }

        .damage-popup {
            position: absolute;
            /* MODIFIED: Scaled down font */
            font-size: 24px;
            font-weight: bold;
            color: red;
            text-shadow: 0 0 5px black;
            animation: fadeUp 1s ease-out;
            pointer-events: none;
            z-index: 999;
        }

        @keyframes fadeUp {
            from {
                opacity: 1;
                transform: translateY(0);
            }
            to {
                opacity: 0;
                /* MODIFIED: Scaled down animation */
                transform: translateY(-40px);
            }
        }

        /* --- GAME OVER MODAL STYLES --- */
        .modal-overlay {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background-color: rgba(0, 0, 0, 0.85);
            display: flex;
            justify-content: center;
            align-items: center;
            z-index: 1000;
        }

        .modal-overlay.hidden {
            display: none;
        }

        .modal-content {
            background: #2c2c2c;
            /* MODIFIED: Scaled down padding */
            padding: 32px;
            border-radius: 10px;
            text-align: center;
            border: 2px solid #111;
            box-shadow: 0 0 20px rgba(0,0,0,0.5);
        }

        .modal-content h2 {
            /* MODIFIED: Scaled down font */
            font-size: 39px;
            margin-top: 0;
            /* MODIFIED: Scaled down margin */
            margin-bottom: 16px;
        }

        #play-again-button {
            /* MODIFIED: Scaled down padding/font */
            padding: 10px 20px;
            font-size: 16px;
            font-weight: bold;
            color: #fff;
            background-color: #27ae60;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.2s;
        }
        #play-again-button:hover {
            background-color: #2ecc71;
        }
    </style>
</head>
<body>

    <div id="game-message">Loading Game...</div>

    <div id="game-container">
        <div class="health-display" id="ai-health">
            <div class="health-bar-inner" id="ai-health-bar"></div>
            <span class="health-text" id="ai-health-text">20</span>
        </div>

        <div class="hand" id="ai-hand"></div>
        <div class="board" id="ai-board"></div>

        <div class="board" id="player-board"></div>

        <div class="health-display" id="player-health">
            <div class="health-bar-inner" id="player-health-bar"></div>
            <span class="health-text" id="player-health-text">20</span>
        </div>
    </div>

    <div id="player-hand-container">
        <div class="hand" id="player-hand"></div>
    </div>

    <div id="game-controls">
        <button id="end-turn-button" disabled>End Turn</button>
    </div>

    <div id="game-over-modal" class="modal-overlay hidden">
        <div class="modal-content">
            <h2 id="game-over-message">You Win!</h2>
            <button id="play-again-button">Play Again</button>
        </div>
    </div>

    <script>
        // --- 1. CARD DATA with public image URLs ---
        const cardData = [
            { "id": "slime_001", "name": "Tiny Slime", "art": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT3VGwpkxZOhvNj1OONh0pL4HeBQLVdni7MKg&s", "attack": 1, "health": 1 },
            { "id": "goblin_001", "name": "Goblin Scout", "art": "https://cdnb.artstation.com/p/assets/images/images/020/458/659/large/karen-petrasko-goblinscout.jpg?1567848564", "attack": 2, "health": 1 },
            { "id": "rat_001", "name": "Dire Rat", "art": "https://i.pinimg.com/736x/a4/3f/e1/a43fe1337170264e2d8b2e1be49e86f7.jpg", "attack": 1, "health": 2 },
            { "id": "skeleton_001", "name": "Shambling Skeleton", "art": "https://cdn.thingiverse.com/assets/21/bb/01/77/91/large_display_skelleton.jpg", "attack": 2, "health": 2 },
            { "id": "orc_001", "name": "Orc Grunt", "art": "https://warcraft.wiki.gg/images/Orgrimmar_Grunt_Thorns.jpg?7f9bdb", "attack": 3, "health": 2 },
            { "id": "wolf_001", "name": "Shadow Wolf", "art": "https://i.pinimg.com/564x/be/23/26/be232652356d11566de0564d69b14bbf.jpg", "attack": 2, "health": 3 },
            { "id": "golem_001", "name": "Stone Golem", "art": "https://makerworld.bblmw.com/makerworld/model/USfc68b249d0f385/design/2025-06-27_7e2c75e71d5978.jpeg?x-oss-process=image/resize,w_1000/format,webp", "attack": 1, "health": 10 },
            { "id": "spider_001", "name": "Gloom Spider", "art": "https://lotro-wiki.com/images/thumb/c/cc/Gloom-web_Spiderling.jpg/800px-Gloom-web_Spiderling.jpg", "attack": 4, "health": 2 },
            { "id": "mage_001", "name": "Imp Familiar", "art": "https://bg3.wiki/w/images/2/20/Imp_Model.png", "attack": 3, "health": 3 },
            { "id": "troll_001", "name": "Rock Troll", "art": "https://www.miniaturemarket.com/media/1b/eb/94/1752267456/nlmdh2041.jpg", "attack": 2, "health": 5 },
            { "id": "ogre_001", "name": "Cave Ogre", "art": "https://img.freepik.com/premium-photo/ogre-cave_1040433-31847.jpg", "attack": 4, "health": 5 },
            { "id": "hydra_001", "name": "Swamp Hydra", "art": "https://imgcdn.stablediffusionweb.com/2024/4/23/475c4df5-211e-4108-ba14-4368f3231484.jpg", "attack": 5, "health": 10 },
            { "id": "elemental_001", "name": "Rock Elemental", "art": "https://i.pinimg.com/474x/25/d6/10/25d610783fc816c1fe78e93c5005b8c2.jpg", "attack": 3, "health": 12 },
            { "id": "demon_001", "name": "Lesser Demon", "art": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRmT4G9CDeUhaEBws-v8fu-oFkH993f4d-egw&s", "attack": 4, "health": 4 },
            { "id": "dragon_001", "name": "Young Dragon", "art": "https://i.etsystatic.com/49604189/r/il/0645f3/6113693146/il_fullxfull.6113693146_g5gk.jpg", "attack": 5, "health": 4 }
        ];

        // --- 2. GAME STATE ---
        const INITIAL_GAME_STATE = {
            playerHealth: 20,
            playerMaxHealth: 20,
            aiHealth: 20,
            aiMaxHealth: 20,
            playerDeck: [],
            aiDeck: [],
            playerHand: [],
            aiHand: [],
            playerBoard: [],
            aiBoard: [],
            turn: 'player',
            selectedAttacker: null,
            gameIsOver: false,
            playerHasPlayedCard: false,
        };
        let gameState;

        // --- 3. DOM SELECTORS ---
        const ui = {
            playerHealthText: document.getElementById('player-health-text'),
            playerHealthBar: document.getElementById('player-health-bar'),
            aiHealthText: document.getElementById('ai-health-text'),
            aiHealthBar: document.getElementById('ai-health-bar'),

            playerHand: document.getElementById('player-hand'),
            aiHand: document.getElementById('ai-hand'),
            playerBoard: document.getElementById('player-board'),
            aiBoard: document.getElementById('ai-board'),
            endTurnButton: document.getElementById('end-turn-button'),
            gameMessage: document.getElementById('game-message'),
            aiHealthClickTarget: document.getElementById('ai-health'),
            playerHealthClickTarget: document.getElementById('player-health'),

            gameOverModal: document.getElementById('game-over-modal'),
            gameOverMessage: document.getElementById('game-over-message'),
            playAgainButton: document.getElementById('play-again-button')
        };

        // --- 4. UTILITY FUNCTIONS ---
        function shuffle(array) {
            for (let i = array.length - 1; i > 0; i--) {
                const j = Math.floor(Math.random() * (i + 1));
                [array[i], array[j]] = [array[j], array[i]];
            }
            return array;
        }

        const delay = ms => new Promise(res => setTimeout(res, ms));

        function showDamage(amount, element) {
            const popup = document.createElement('div');
            popup.className = 'damage-popup';
            popup.textContent = `-${amount}`;
            document.body.appendChild(popup);

            const rect = element.getBoundingClientRect();
            popup.style.left = `${rect.left + rect.width / 2 - 15}px`;
            popup.style.top = `${rect.top + rect.height / 2 - 15}px`;

            setTimeout(() => popup.remove(), 1000);
        }

        // --- 5. RENDER FUNCTIONS ---

        function createCardElement(cardInstance, isAIHand = false) {
            const cardEl = document.createElement('div');
            cardEl.className = 'card';
            cardEl.dataset.instanceId = cardInstance.instanceId;

            if (cardInstance.hasAttacked) {
                cardEl.classList.add('exhausted');
            }

            if (isAIHand) {
                cardEl.classList.add('card-back');
            } else {
                cardEl.style.backgroundImage = `url('${cardInstance.base.art}')`;
                cardEl.innerHTML = `
                    <div class="card-name">${cardInstance.base.name}</div>
                    <div class="card-attack">${cardInstance.currentAttack}</div>
                    <div class="card-health">${cardInstance.currentHealth}</div>
                `;
            }
            return cardEl;
        }

        function render() {
            if (gameState.gameIsOver && !ui.gameOverModal.classList.contains('hidden')) return;

            ui.playerHealthText.textContent = gameState.playerHealth;
            const playerHealthPercent = (gameState.playerHealth / gameState.playerMaxHealth) * 100;
            ui.playerHealthBar.style.width = `${playerHealthPercent}%`;

            ui.aiHealthText.textContent = gameState.aiHealth;
            const aiHealthPercent = (gameState.aiHealth / gameState.aiMaxHealth) * 100;
            ui.aiHealthBar.style.width = `${aiHealthPercent}%`;

            ui.playerHand.innerHTML = '';
            gameState.playerHand.forEach(card => {
                const cardEl = createCardElement(card);
                cardEl.addEventListener('click', () => onPlayCard(card));
                ui.playerHand.appendChild(cardEl);
            });

            ui.playerBoard.innerHTML = '';
            gameState.playerBoard.forEach(card => {
                const cardEl = createCardElement(card);
                cardEl.addEventListener('click', () => onSelectAttacker(card, cardEl));
                ui.playerBoard.appendChild(cardEl);
            });

            ui.aiBoard.innerHTML = '';
            gameState.aiBoard.forEach(card => {
                const cardEl = createCardElement(card);
                cardEl.addEventListener('click', () => onSelectTarget(card));
                ui.aiBoard.appendChild(cardEl);
            });

            ui.aiHand.innerHTML = '';
            for (let i = 0; i < gameState.aiHand.length; i++) {
                const cardEl = createCardElement({}, true);
                ui.aiHand.appendChild(cardEl);
            }
        }

        // --- 6. GAME LOGIC ---

        function drawCard(playerType, count = 1) {
            for (let i = 0; i < count; i++) {
                const deck = gameState[`${playerType}Deck`];
                const hand = gameState[`${playerType}Hand`];
                if (deck.length > 0) {
                    const cardData = deck.pop();
                    const cardInstance = {
                        base: cardData,
                        instanceId: `${playerType}_${Math.random().toString(36).substr(2, 9)}`,
                        currentHealth: cardData.health,
                        currentAttack: cardData.attack,
                        hasAttacked: false
                    };
                    hand.push(cardInstance);
                } else {
                    console.log(`${playerType} is out of cards!`);
                }
            }
        }

        function onPlayCard(card) {
            if (gameState.turn !== 'player' || gameState.playerHasPlayedCard || gameState.gameIsOver) return;

            if (gameState.playerBoard.length >= 5) {
                updateMessage("Your board is full!");
                return;
            }

            gameState.playerHand = gameState.playerHand.filter(c => c.instanceId !== card.instanceId);
            gameState.playerBoard.push(card);
            gameState.playerHasPlayedCard = true;
            render();
        }

        function onSelectAttacker(card, element) {
            if (gameState.turn !== 'player' || gameState.gameIsOver) return;

            if (card.hasAttacked) {
                updateMessage("This creature has already attacked!");
                return;
            }

            if (gameState.selectedAttacker) {
                gameState.selectedAttacker.element.classList.remove('selected');
            }

            gameState.selectedAttacker = { card, element };
            element.classList.add('selected');
        }

        function onSelectTarget(targetCard) {
            if (!gameState.selectedAttacker || gameState.gameIsOver) return;

            const attacker = gameState.selectedAttacker.card;

            const targetElement = ui.aiBoard.querySelector(`[data-instance-id="${targetCard.instanceId}"]`);

            handleAttack(attacker, targetCard, targetElement);

            attacker.hasAttacked = true;

            gameState.selectedAttacker.element.classList.remove('selected');
            gameState.selectedAttacker = null;
            render();
        }

        function handleAttack(attacker, target, targetElement) {
            if (gameState.gameIsOver) return;

            if (target === 'aiHero') {
                gameState.aiHealth -= attacker.currentAttack;
                gameState.aiHealth = Math.max(0, gameState.aiHealth);
                showDamage(attacker.currentAttack, targetElement);
            } else if (target === 'playerHero') {
                gameState.playerHealth -= attacker.currentAttack;
                gameState.playerHealth = Math.max(0, gameState.playerHealth);
                showDamage(attacker.currentAttack, targetElement);
            } else {
                target.currentHealth -= attacker.currentAttack;
                showDamage(attacker.currentAttack, targetElement);

                if (target.currentHealth <= 0) {
                    if (gameState.playerBoard.includes(target)) {
                        gameState.playerBoard = gameState.playerBoard.filter(c => c.instanceId !== target.instanceId);
                    } else {
                        gameState.aiBoard = gameState.aiBoard.filter(c => c.instanceId !== target.instanceId);
                    }
                }
            }

            checkGameOver();
        }

        function checkGameOver() {
            if (gameState.gameIsOver) return;

            let winner = null;
            if (gameState.aiHealth <= 0) {
                winner = 'player';
            } else if (gameState.playerHealth <= 0) {
                winner = 'ai';
            }

            if (winner) {
                gameState.gameIsOver = true;
                ui.endTurnButton.disabled = true;
                ui.gameOverMessage.textContent = (winner === 'player') ? "YOU WIN!" : "YOU LOSE!";
                ui.gameOverModal.classList.remove('hidden');
                updateMessage("");
            }
        }

        // --- 7. AI LOGIC ---

        async function aiTurn() {
            updateMessage("AI is thinking...");
            await delay(1000);

            // Draw AI cards up to 4 at start of turn
            while (gameState.aiHand.length < 4 && gameState.aiDeck.length > 0) {
                drawCard('ai', 1);
            }

            if (gameState.aiBoard.length < 5 && gameState.aiHand.length > 0) {
                let cardToPlay = gameState.aiHand.sort((a, b) => b.base.attack - a.base.attack)[0];
                gameState.aiHand = gameState.aiHand.filter(c => c.instanceId !== cardToPlay.instanceId);
                gameState.aiBoard.push(cardToPlay);
                updateMessage("AI plays a card!");
                render();
                await delay(1000);
            }

            updateMessage("AI attacks!");
            await delay(500);

            for (const aiCreature of gameState.aiBoard) {
                if (aiCreature.hasAttacked) continue;

                let targetCreature = gameState.playerBoard.find(pc => pc.currentHealth <= aiCreature.currentAttack);

                if (targetCreature) {
                    const targetElement = ui.playerBoard.querySelector(`[data-instance-id="${targetCreature.instanceId}"]`);
                    handleAttack(aiCreature, targetCreature, targetElement);
                } else {
                    handleAttack(aiCreature, "playerHero", ui.playerHealthClickTarget);
                }

                aiCreature.hasAttacked = true;

                render();
                if (gameState.gameIsOver) break;
                await delay(750);
            }

            if (gameState.gameIsOver) return;

            await delay(1000);
            drawCard('player', 1);
            gameState.turn = 'player';
            gameState.playerHasPlayedCard = false;

            gameState.playerBoard.forEach(card => card.hasAttacked = false);

            ui.endTurnButton.disabled = false;
            updateMessage("Your turn!");
            render();
        }

        function endPlayerTurn() {
            if (gameState.turn !== 'player' || gameState.gameIsOver) return;

            gameState.turn = 'ai';
            ui.endTurnButton.disabled = true;

            if (gameState.selectedAttacker) {
                gameState.selectedAttacker.element.classList.remove('selected');
                gameState.selectedAttacker = null;
            }

            gameState.aiBoard.forEach(card => card.hasAttacked = false);

            aiTurn();
        }

        function updateMessage(msg) {
            ui.gameMessage.textContent = msg;
        }

        // --- 8. START/RESET GAME ---

        function resetGame() {
            gameState = JSON.parse(JSON.stringify(INITIAL_GAME_STATE));

            ui.playerHand.innerHTML = '';
            ui.aiHand.innerHTML = '';
            ui.playerBoard.innerHTML = '';
            ui.aiBoard.innerHTML = '';

            ui.gameOverModal.classList.add('hidden');

            gameState.playerDeck = [...cardData, ...cardData];
            gameState.aiDeck = [...cardData, ...cardData];

            shuffle(gameState.playerDeck);
            shuffle(gameState.aiDeck);

            drawCard('player', 4);
            drawCard('ai', 4);

            ui.endTurnButton.disabled = false;
            updateMessage("Your turn!");

            render();
        }

        function init() {
            ui.endTurnButton.addEventListener('click', endPlayerTurn);
            ui.playAgainButton.addEventListener('click', resetGame);

            ui.aiHealthClickTarget.addEventListener('click', () => {
                if (!gameState.selectedAttacker || gameState.gameIsOver) return;

                if (gameState.selectedAttacker.card.hasAttacked) {
                    updateMessage("This creature has already attacked!");
                    return;
                }

                const attacker = gameState.selectedAttacker.card;

                handleAttack(attacker, "aiHero", ui.aiHealthClickTarget);

                attacker.hasAttacked = true;

                gameState.selectedAttacker.element.classList.remove('selected');
                gameState.selectedAttacker = null;
                render();
            });

            resetGame();
        }

        init();
    </script>
</body>
</html>"""